In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

df1=pd.read_csv('D:/esg/esg2/基本信息/基本信息_esg.csv', encoding='ANSI')
df2=pd.read_csv('D:/esg/esg2/经营能力/jingyingnengli_esg.csv', encoding='ANSI')
df3=pd.read_csv('D:/esg/esg2/偿债能力/changzhainengli_esg.csv', encoding='ANSI')
df4=pd.read_csv('D:/esg/esg2/盈利能力/yinglinengli_esg.csv', encoding='ANSI')
df5=pd.read_csv('D:/esg/esg2/发展能力/fazhannengli_esg.csv', encoding='ANSI')
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)


Index(['year', 'code', '股利支付率', '股权性质', '企业规模', '托宾Q值A', '账面市值比A', 'ESG'], dtype='object')
Index(['code', 'year', '应收账款与收入比', '存货与收入比', '存货周转率A', '营业周期A', '应付账款周转率A',
       '营运资金(资本)周转率A', '流动资产周转率A', '固定资产周转率A', '总资产周转率A', '股东权益周转率A', 'ESG'],
      dtype='object')
Index(['code', 'year', '现金比率', '营运资金', '利息保障倍数A', '经营活动产生的现金流量净额／流动负债',
       '资产负债率', '有形资产带息债务比', '权益乘数', '经营活动产生的现金流量净额／负债合计', 'ESG', '流动比率',
       '速动比率'],
      dtype='object')
Index(['code', 'year', '总资产净利润率(ROA)A', '流动资产净利润率A', '固定资产净利润率A',
       '净资产收益率（ROE）A', '净利润与利润总额比', '投入资本回报率（ROIC）', '营业净利率', '成本费用利润率',
       '资产减值损失／营业收入', '长期资本收益率', 'ESG'],
      dtype='object')
Index(['code', 'year', '资本保值增值率A', '资本积累率A', '固定资产增长率A', '总资产增长率A', '综合收益增长率',
       '营业收入增长率A', '营业总收入增长率', '可持续增长率', '所有者权益增长率A', '每股净资产增长率A', 'ESG'],
      dtype='object')


In [8]:
df_jiben_jingying = pd.merge(df1, df2[['code', 'year','应收账款与收入比', '存货与收入比', '存货周转率A', '营业周期A', '应付账款周转率A',
       '营运资金(资本)周转率A', '流动资产周转率A', '固定资产周转率A', '总资产周转率A', '股东权益周转率A']], on=['code', 'year'], how='left') 
df_jingying_changzhai=pd.merge(df_jiben_jingying, df3[['code', 'year','现金比率', '营运资金', '利息保障倍数A', '经营活动产生的现金流量净额／流动负债',
       '资产负债率', '有形资产带息债务比', '权益乘数', '经营活动产生的现金流量净额／负债合计',  '流动比率',
       '速动比率']], on=['code', 'year'], how='left') 
df_jingying_changzhai_fazhan=pd.merge(df_jingying_changzhai, df5[['code', 'year','资本保值增值率A', '资本积累率A', '固定资产增长率A', '总资产增长率A', '综合收益增长率',
       '营业收入增长率A', '营业总收入增长率', '可持续增长率', '所有者权益增长率A', '每股净资产增长率A']], on=['code', 'year'], how='left') 
data1=pd.merge(df_jingying_changzhai_fazhan, df4[['code', 'year','总资产净利润率(ROA)A', '流动资产净利润率A', '固定资产净利润率A',
       '净资产收益率（ROE）A', '净利润与利润总额比', '投入资本回报率（ROIC）', '营业净利率', '成本费用利润率',
       '资产减值损失／营业收入', '长期资本收益率']], on=['code', 'year'], how='left') 

In [9]:
print(data1.columns)

Index(['year', 'code', '股利支付率', '股权性质', '企业规模', '托宾Q值A', '账面市值比A', 'ESG',
       '应收账款与收入比', '存货与收入比', '存货周转率A', '营业周期A', '应付账款周转率A', '营运资金(资本)周转率A',
       '流动资产周转率A', '固定资产周转率A', '总资产周转率A', '股东权益周转率A', '现金比率', '营运资金',
       '利息保障倍数A', '经营活动产生的现金流量净额／流动负债', '资产负债率', '有形资产带息债务比', '权益乘数',
       '经营活动产生的现金流量净额／负债合计', '流动比率', '速动比率', '资本保值增值率A', '资本积累率A', '固定资产增长率A',
       '总资产增长率A', '综合收益增长率', '营业收入增长率A', '营业总收入增长率', '可持续增长率', '所有者权益增长率A',
       '每股净资产增长率A', '总资产净利润率(ROA)A', '流动资产净利润率A', '固定资产净利润率A', '净资产收益率（ROE）A',
       '净利润与利润总额比', '投入资本回报率（ROIC）', '营业净利率', '成本费用利润率', '资产减值损失／营业收入',
       '长期资本收益率'],
      dtype='object')


In [18]:
def observe_performance(data):
    aa=0
    X1 = data.drop(['ESG','code','year'], axis=1)
    y = data['ESG']
    X = X1.dropna()
    y = y[X.index]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X=pd.DataFrame(X)  
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=2)
    rf_model = RandomForestRegressor(n_estimators=150, random_state=0)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    r2_test = r2_score(y_test, y_pred)
    print(f"样本外拟合优度（测试集）: {r2_test}")
    mse = mean_squared_error(y_test, y_pred)
    print(f"均方误差: {mse}")
    feature_names = [6,7,8,9,10,11,12,13,14,15]#经营能力
    feature_names2 = [15,16,17,18,19,20,21,22,23,24]#偿债能力
    feature_names3 = [25,26,27,28,29,30,31,32,33,34]#盈利能力
    feature_names4 = [35,36,37,38,39,40,41,42,43,44]#发展能力
    importances = rf_model.feature_importances_
    for feature in feature_names:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    print('---------------')
    aa=0
    for feature in feature_names2:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    print('---------------')
    aa=0
    for feature in feature_names3:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    print('---------------')
    aa=0
    for feature in feature_names4:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    
observe_performance(data1)    

样本外拟合优度（测试集）: 0.36816935343238877
均方误差: 0.7861077632183908
存货与收入比
0.01696319886888645
存货周转率A
0.019196965952473288
营业周期A
0.017229247037885365
应付账款周转率A
0.022292185343795186
营运资金(资本)周转率A
0.01466109100538513
流动资产周转率A
0.01435024604691729
固定资产周转率A
0.017663580618750993
总资产周转率A
0.013652667940223049
股东权益周转率A
0.01386888733489737
现金比率
0.020826847574981353
0.17070491772419547
---------------
现金比率
0.020826847574981353
营运资金
0.04578002069077688
利息保障倍数A
0.019955360101523496
经营活动产生的现金流量净额／流动负债
0.015266477228155485
资产负债率
0.010473404015687757
有形资产带息债务比
0.025329009506449233
权益乘数
0.01003816338977657
经营活动产生的现金流量净额／负债合计
0.014702581231603859
流动比率
0.015366138901782763
速动比率
0.015269736392372188
0.19300773903310958
---------------
资本保值增值率A
0.04952614361290604
资本积累率A
0.04305589534529565
固定资产增长率A
0.028563342438706153
总资产增长率A
0.0206301770115601
综合收益增长率
0.026520407042889604
营业收入增长率A
0.021517393457343822
营业总收入增长率
0.023378704394610327
可持续增长率
0.015456712044709746
所有者权益增长率A
0.04920366799376486
每股净资产增长率A
0.03075759835037

In [20]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score
import numpy as np

plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 
def observe_performance_xgboost(data):
    aa=0
    X1 = data.drop(['ESG','code','year'], axis=1)
    y = data['ESG']
    X = X1.dropna()
    y = y[X.index]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X=pd.DataFrame(X)  
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=2)
    xgb_model = xgb.XGBRegressor(n_estimators=150, random_state=0,reg_alpha=5, reg_lambda=5)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"样本外均方误差: {mse}")
    y_pred_train = xgb_model.predict(X_train)
    r2_train = r2_score(y_train, y_pred_train)
    print(f"样本内拟合优度（训练集）: {r2_train}")
    r2_test = r2_score(y_test, y_pred)
    print(f"样本外拟合优度（测试集）: {r2_test}")
    importances = xgb_model.feature_importances_
    feature_names = [6,7,8,9,10,11,12,13,14,15]#经营能力
    feature_names2 = [15,16,17,18,19,20,21,22,23,24]#偿债能力
    feature_names3 = [25,26,27,28,29,30,31,32,33,34]#盈利能力
    feature_names4 = [35,36,37,38,39,40,41,42,43,44]#发展能力
    for feature in feature_names:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    aa=0
    print('---------------')    
    for feature in feature_names2:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    print('---------------')
    aa=0
    for feature in feature_names3:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
    print('---------------')
    aa=0
    for feature in feature_names4:
        print(X1.columns[feature])
        print(importances[feature])
        aa=aa+importances[feature]
    print(aa)
observe_performance_xgboost(data1)

样本外均方误差: 0.8424390284522042
样本内拟合优度（训练集）: 0.9226165096526256
样本外拟合优度（测试集）: 0.3228933475207615
存货与收入比
0.019189369
存货周转率A
0.028266674
营业周期A
0.020410191
应付账款周转率A
0.016702956
营运资金(资本)周转率A
0.015174659
流动资产周转率A
0.016476983
固定资产周转率A
0.013337604
总资产周转率A
0.01598747
股东权益周转率A
0.014894101
现金比率
0.015624386
0.17606439348310232
---------------
现金比率
0.015624386
营运资金
0.026730608
利息保障倍数A
0.01600932
经营活动产生的现金流量净额／流动负债
0.011929991
资产负债率
0.017511675
有形资产带息债务比
0.018299244
权益乘数
0.010538142
经营活动产生的现金流量净额／负债合计
0.014813647
流动比率
0.01728646
速动比率
0.01762535
0.16636882442981005
---------------
资本保值增值率A
0.16472745
资本积累率A
0.0
固定资产增长率A
0.020027664
总资产增长率A
0.014419454
综合收益增长率
0.019457774
营业收入增长率A
0.014657102
营业总收入增长率
0.014710439
可持续增长率
0.014802928
所有者权益增长率A
0.0
每股净资产增长率A
0.027986519
0.2907893294468522
---------------
总资产净利润率(ROA)A
0.02084435
流动资产净利润率A
0.014996566
固定资产净利润率A
0.013057343
净资产收益率（ROE）A
0.018688593
净利润与利润总额比
0.018630037
投入资本回报率（ROIC）
0.017890874
营业净利率
0.019374609
成本费用利润率
0.018514188
资产减值损失／营业收入
0.020058783
长